# Building an image retrieval system with deep features


# Fire up GraphLab Create

In [1]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [2]:
image_train = graphlab.SFrame('image_train_data/')

This non-commercial license of GraphLab Create for academic use is assigned to yu.sue.liu@cantab.net and will expire on April 01, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1495178153.log


# Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [ ]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [3]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [4]:
image_train_bird = image_train[image_train['label'] == 'bird']
image_train_cat = image_train[image_train['label'] == 'cat']
image_train_dog = image_train[image_train['label'] == 'dog']
image_train_car = image_train[image_train['label'] == 'automobile']

In [5]:
bird_model = graphlab.nearest_neighbors.create(image_train_bird, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

In [6]:
cat_model = graphlab.nearest_neighbors.create(image_train_cat, features=['deep_features'], label='id')
dog_model = graphlab.nearest_neighbors.create(image_train_dog, features=['deep_features'], label='id')
car_model = graphlab.nearest_neighbors.create(image_train_car, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [7]:
image_test = graphlab.SFrame('image_test_data/')

In [11]:
graphlab.canvas.set_target('ipynb')
cat1 = image_test[0:1]
cat1['image'].show()
cat_model.query(cat1)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 22.447ms     |

| Done         |         | 100         | 83.369ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [ ]:
dog_model.query(cat1)

In [ ]:
image_cat_train[image_cat_train['id'] == 16289]['image'].show()

# Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [ ]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

## Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [ ]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [ ]:
knn_model.query(cat)

We are going to create a simple function to view the nearest neighbors to save typing:

In [ ]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [ ]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

In [ ]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

## Finding similar images to a car

In [ ]:
car = image_train[8:9]
car['image'].show()

In [ ]:
get_images_from_ids(knn_model.query(car))['image'].show()

#Just for fun, let's create a lambda to find and show nearest neighbor images

In [ ]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [ ]:
show_neighbors(8)

In [ ]:
show_neighbors(26)